# Transducer PyTorch implementation
Goal: In this notebook, we'll use it to insert missing vowels into a sentence.

This is an image of the architecture.

![RNN-T architecture](images/rnnt_architecture.png)

In [1]:
# Get training data.
!wget https://raw.githubusercontent.com/lorenlugosch/infer_missing_vowels/master/data/train/war_and_peace.txt

--2023-05-06 08:21:22--  https://raw.githubusercontent.com/lorenlugosch/infer_missing_vowels/master/data/train/war_and_peace.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3196229 (3.0M) [text/plain]
Saving to: ‘war_and_peace.txt.1’

war_and_peace.txt.1 100%[===================>]   3.05M  4.39MB/s    in 0.7s    

2023-05-06 08:21:24 (4.39 MB/s) - ‘war_and_peace.txt.1’ saved [3196229/3196229]



In [10]:
# Imports
import torch
from tqdm import tqdm
from torch import nn
import torch.nn.functional as F
import itertools
from collections import Counter
import IPython
import string


# Variables
NULL_INDEX = 0
encoder_dim = 1024
predictor_dim = 1024
joiner_dim = 1024
     

In [8]:
# Encoder network
# The encoder is any network that can take as input a variable-length sequence: so, RNNs, CNNs, and self-attention/Transformer encoders will all work.

class Encoder(nn.Module):
    """Encoder network.
    
    Input: Audio input

    Output: Encoded speech features. Context-aware encodings.
    """
    def __init__(self, num_inputs):
        super(Encoder, self).__init__()
        self.embed = torch.nn.Embedding(num_inputs, encoder_dim)
        self.rnn = torch.nn.GRU(input_size=encoder_dim, hidden_size=encoder_dim, num_layers=3, batch_first=True, bidirectional=True, dropout=0.1)
        self.linear = torch.nn.Linear(encoder_dim*2, joiner_dim)

    def forward(self, x):
        out = x
        out = self.embed(out)
        out = self.rnn(out)[0]
        out = self.linear(out)
        return out

In [ ]:
# Predictor network
# The predictor is any causal network (= can't look at the future): in other words, unidirectional RNNs, causal convolutions, or masked self-attention.

class Predictor(torch.nn.Module):
    """Predictor network.
    
    Input: Text inputs (labels).
    
    Output: RNN hidden states for each autoregressive timestep input.
    """
    def __init__(self, num_outputs):
        super(Predictor, self).__init__()
        self.embed = torch.nn.Embedding(num_outputs, predictor_dim)
        self.rnn = torch.nn.GRUCell(input_size=predictor_dim, hidden_size=predictor_dim)
        self.linear = torch.nn.Linear(predictor_dim, joiner_dim)
        
        self.initial_state = torch.nn.Parameter(torch.randn(predictor_dim))
        self.start_symbol = NULL_INDEX # In the original paper, a vector of 0s is used; just using the null index instead is easier when using an Embedding layer.

    def forward_one_step(self, input, previous_state):
        embedding = self.embed(input)
        state = self.rnn.forward(embedding, previous_state)
        out = self.linear(state)
        return out, state

    def forward(self, y):
        batch_size = y.shape[0]
        U = y.shape[1]
        outs = []
        state = torch.stack([self.initial_state] * batch_size).to(y.device)
        for u in range(U+1): # need U+1 to get null output for final timestep 
            if u == 0:
                decoder_input = torch.tensor([self.start_symbol] * batch_size).to(y.device)
            else:
                decoder_input = y[:,u-1]
            out, state = self.forward_one_step(decoder_input, state)
            outs.append(out)
        out = torch.stack(outs, dim=1)
        return out

In [ ]:
class Joiner(torch.nn.Module):
    def __init__(self, num_outputs):
        super(Joiner, self).__init__()
        self.linear = torch.nn.Linear(joiner_dim, num_outputs)

    def forward(self, encoder_out, predictor_out):
        """Forward pass.
        
        Args:
            encoder_out: Output of the encoder network. Shape [batch_size, T, 1, encoder_dim]. # TODO Confirm that these shapes are correct.
            predictor_out: Output of the predictor network. Shape [batch_size, 1, U, predictor_dim]. # TODO Confirm that these shapes are correct.
        """
        out = encoder_out + predictor_out
        out = torch.nn.functional.relu(out)
        out = self.linear(out)
        return out

Alignment matrix

![RNN-T alignment matrix](images/rnnt_alignment_matrix.png)

In [14]:
class Transducer(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Transducer, self).__init__()
        self.encoder = Encoder(num_inputs)
        self.predictor = Predictor(num_outputs)
        self.joiner = Joiner(num_outputs)

        if torch.cuda.is_available():
            self.device = "cuda:0"
        else:
            self.device = "cpu"
        self.to(self.device)


    def compute_forward_prob(self, joiner_out, T, U, y):
        """Compute forward probability.

        Args:
            joiner_out: tensor of shape (B, T_max, U_max+1, #labels)
            T: list of input lengths
            U: list of output lengths 
            y: label tensor (B, U_max+1)

        Returns:
            log_probs: tensor of shape (B)
        """
        # TODO understand this
        B = joiner_out.shape[0]
        T_max = joiner_out.shape[1]
        U_max = joiner_out.shape[2] - 1
        log_alpha = torch.zeros(B, T_max, U_max+1).to(self.device)
        for t in range(T_max):
            for u in range(U_max+1):
                if u == 0:
                    if t == 0:
                        log_alpha[:, t, u] = 0.

                    else: #t > 0
                        log_alpha[:, t, u] = log_alpha[:, t-1, u] + joiner_out[:, t-1, 0, NULL_INDEX] 
                            
                else: #u > 0
                    if t == 0:
                        log_alpha[:, t, u] = log_alpha[:, t,u-1] + torch.gather(joiner_out[:, t, u-1], dim=1, index=y[:,u-1].view(-1,1) ).reshape(-1)
                    
                    else: #t > 0
                        log_alpha[:, t, u] = torch.logsumexp(torch.stack([
                            log_alpha[:, t-1, u] + joiner_out[:, t-1, u, NULL_INDEX],
                            log_alpha[:, t, u-1] + torch.gather(joiner_out[:, t, u-1], dim=1, index=y[:,u-1].view(-1,1) ).reshape(-1)
                        ]), dim=0)

        log_probs = []
        for b in range(B):
            log_prob = log_alpha[b, T[b]-1, U[b]] + joiner_out[b, T[b]-1, U[b], NULL_INDEX]
            log_probs.append(log_prob)
        log_probs = torch.stack(log_probs) 
        return log_prob

    def compute_loss(self, x, y, T, U):
        encoder_out = self.encoder.forward(x)
        predictor_out = self.predictor.forward(y)
        joiner_out = self.joiner.forward(encoder_out.unsqueeze(2), predictor_out.unsqueeze(1)).log_softmax(3)
        loss = -self.compute_forward_prob(joiner_out, T, U, y).mean()
        return loss
  
    def compute_single_alignment_prob(self, encoder_out, predictor_out, T, U, z, y):
        """Computes the probability of one alignment, z.

        What does the encoder_out network represent?
        What does the predictor_out network represent?


        Args:
            encoder_out: Output of the encoder network. Shape [T, encoder_dim]. T = 4. In this example it is [4, 1024]
            predictor_out: Output of the predictor network. Shape [U, predictor_dim]. U = 3+1 = 4. In this example it is [4, 1024]
            T: Length of the encoder_out sequence.
            U: Length of the predictor_out sequence.
            z: Alignment. List of 0s and 1s, where 0 means "right" (increment t, encoder output) and 1 means "down" (increment u, label).
            y: Label sequence. List of labels.
        
        Returns:
            logprob: Log probability of the alignment.
        """
        t = 0; u = 0 # t and u are merely movements in the alignment matrix that we'll use to index into the encoder_out and predictor_out matrices.
        t_u_indices = []
        y_expanded = []
        for step in z:
            t_u_indices.append((t,u))
            if step == 0: # right (null)
                y_expanded.append(NULL_INDEX)
                t += 1
            if step == 1: # down (label)
                y_expanded.append(y[u])
                u += 1

        t_u_indices.append((T-1,U))
        y_expanded.append(NULL_INDEX)

        t_indices = [t for (t,u) in t_u_indices]
        u_indices = [u for (t,u) in t_u_indices]
        encoder_out_expanded = encoder_out[t_indices] # Indexed into the encoder/predictor_out lists using the t_u alignments.
        predictor_out_expanded = predictor_out[u_indices]
        joiner_out = self.joiner.forward(encoder_out_expanded, predictor_out_expanded).log_softmax(1) # joiner network yoinks the alignments at each timestep ((0, 0) hidden states) through it and pops out softmaxed probabilities.
        logprob = -torch.nn.functional.nll_loss(input=joiner_out, target=torch.tensor(y_expanded).long().to(self.device), reduction="sum") # TODO HERE!!!!!! Why is the target expanded?
        IPython.embed()
        return logprob  

In [15]:

# Generate example inputs/outputs
num_outputs = len(string.ascii_uppercase) + 1 # [null, A, B, ... Z]
model = Transducer(1, num_outputs)
y_letters = "CAT"
y = torch.tensor([string.ascii_uppercase.index(l) + 1 for l in y_letters]).unsqueeze(0).to(model.device) # tokenize
T = torch.tensor([4]) # Time
U = torch.tensor([len(y_letters)]) # Number of labels
B = 1 # Batch

# encoder_out and predictor_out have the same dimentions.
encoder_out = torch.randn(B, T, joiner_dim).to(model.device) # Fairly sure T = 4 because of NULL_INDEX (0) token prepended to the input. TODO: confirm this.
predictor_out = torch.randn(B, U+1, joiner_dim).to(model.device) # Fairly sure U = 3 + 1 because NULL_INDEX (0) token prepended to the input. TODO: confirm this.
joiner_out = model.joiner.forward(encoder_out.unsqueeze(2), predictor_out.unsqueeze(1)).log_softmax(3)

#######################################################
# Compute loss by enumerating all possible alignments #
#######################################################
all_permutations = list(itertools.permutations([0]*(T-1) + [1]*U))
all_distinct_permutations = list(Counter(all_permutations).keys())
alignment_probs = []
for z in all_distinct_permutations:
    alignment_prob = model.compute_single_alignment_prob(encoder_out[0], predictor_out[0], T.item(), U.item(), z, y[0])
    alignment_probs.append(alignment_prob)
loss_enumerate = -torch.tensor(alignment_probs).logsumexp(0)
IPython.embed()
#######################################################
# Compute loss using the forward algorithm            #
#######################################################
loss_forward = -model.compute_forward_prob(joiner_out, T, U, y)

print("Loss computed by enumerating all possible alignments: ", loss_enumerate)
print("Loss computed using the forward algorithm: ", loss_forward)



Python 3.8.16 (default, Dec  7 2022, 01:27:54) 
Type 'copyright', 'credits' or 'license' for more information
IPython 8.7.0 -- An enhanced Interactive Python. Type '?' for help.

Out[1]: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3)]

Out[2]: [0, 0, 0, tensor(3), tensor(1), tensor(20), 0]

Out[3]: (0, 0, 0, 1, 1, 1)

Out[4]: [0, 1, 2, 3, 3, 3, 3]

Out[5]: [0, 0, 0, 0, 1, 2, 3]

Out[6]: 
tensor([[-0.0871, -0.2618, -1.1075,  ...,  0.0140,  0.1083, -0.2083],
        [-0.8854,  1.5881,  0.6256,  ...,  2.1531, -0.4263, -1.3524],
        [ 0.3115, -0.1987,  0.7019,  ...,  0.1874, -0.4023, -0.4572],
        [ 1.5598,  0.0419,  0.5987,  ...,  0.6928,  0.5698,  0.2840]])

Out[7]: torch.Size([4, 1024])

Out[8]: torch.Size([4, 1024])

Out[9]: 
tensor([[-0.0871, -0.2618, -1.1075,  ...,  0.0140,  0.1083, -0.2083],
        [-0.8854,  1.5881,  0.6256,  ...,  2.1531, -0.4263, -1.3524],
        [ 0.3115, -0.1987,  0.7019,  ...,  0.1874, -0.4023, -0.4572],
        [ 1.5598,  0.0419,  0.5987,  

KeyboardInterrupt: Interrupted by user